In [2]:
library(biomaRt)

In [ ]:
#get all protein coding genes
library(biomaRt)
grch37 = useMart(biomart="ENSEMBL_MART_ENSEMBL", host="grch37.ensembl.org", path="/biomart/martservice", dataset="hsapiens_gene_ensembl")

## get ensembl ID
helper <- function(str){
	unlist(strsplit(str, '\\.'))[1]
}

get_gene_index <- function(chr){
	glist = dir(paste0("/ysm-gpfs/pi/zhao/from_louise/yh367/GTEX/adjusted_expr1/chr", chr));
	tmp = sapply(glist, helper)
	unname(tmp)
}

pc_gene = list()
pc_num = c()
org_num = c()

for(chr in 1:22){
	esd = get_gene_index(chr)
	gene_info = getBM(filters= c("ensembl_gene_id","biotype"), attributes= c("ensembl_gene_id",'hgnc_symbol','chromosome_name'), values= list(esd,"protein_coding"), mart= grch37)
	pc_gene[[chr]] = gene_info
	pc_num = c(pc_num, nrow(gene_info))
	org_num = c(org_num, length(esd))
}
save(pc_gene, pc_num, org_num, file="/ysm-gpfs/pi/zhao/from_louise/yh367/GTEX/protein_coding.RData")

In [4]:
devtools::install_github("stephenturner/annotables")

from URL https://api.github.com/repos/stephenturner/annotables/zipball/master
Installing annotables
Installing tibble
Installing cli
Installing assertthat
'/ysm-gpfs/pi/zhao/Softwares/packages/R/lib64/R/bin/R' --no-site-file  \
  --no-environ --no-save --no-restore --quiet CMD INSTALL  \
  '/tmp/Rtmp811ZCs/devtools6db04ce83272/assertthat'  \
  --library='/gpfs/ysm/home/zy92/R/x86_64-pc-linux-gnu-library/3.5'  \
  --install-tests 

'/ysm-gpfs/pi/zhao/Softwares/packages/R/lib64/R/bin/R' --no-site-file  \
  --no-environ --no-save --no-restore --quiet CMD INSTALL  \
  '/tmp/Rtmp811ZCs/devtools6db04a5b572f/cli'  \
  --library='/gpfs/ysm/home/zy92/R/x86_64-pc-linux-gnu-library/3.5'  \
  --install-tests 

Installing fansi
'/ysm-gpfs/pi/zhao/Softwares/packages/R/lib64/R/bin/R' --no-site-file  \
  --no-environ --no-save --no-restore --quiet CMD INSTALL  \
  '/tmp/Rtmp811ZCs/devtools6db018457d9b/fansi'  \
  --library='/gpfs/ysm/home/zy92/R/x86_64-pc-linux-gnu-library/3.5'  \
  --install-tests 



In [7]:
library(annotables)
library(dplyr)

In [8]:
dim(grch38)

[1] 64428     9

In [9]:
head(grch38)

ensgene,entrez,symbol,chr,start,end,strand,biotype,description
ENSG00000000003,7105,TSPAN6,X,100627109,100639991,-1,protein_coding,tetraspanin 6 [Source:HGNC Symbol;Acc:HGNC:11858]
ENSG00000000005,64102,TNMD,X,100584802,100599885,1,protein_coding,tenomodulin [Source:HGNC Symbol;Acc:HGNC:17757]
ENSG00000000419,8813,DPM1,20,50934867,50958555,-1,protein_coding,"dolichyl-phosphate mannosyltransferase subunit 1, catalytic [Source:HGNC Symbol;Acc:HGNC:3005]"
ENSG00000000457,57147,SCYL3,1,169849631,169894267,-1,protein_coding,SCY1 like pseudokinase 3 [Source:HGNC Symbol;Acc:HGNC:19285]
ENSG00000000460,55732,C1orf112,1,169662007,169854080,1,protein_coding,chromosome 1 open reading frame 112 [Source:HGNC Symbol;Acc:HGNC:25565]
ENSG00000000938,2268,FGR,1,27612064,27635277,-1,protein_coding,"FGR proto-oncogene, Src family tyrosine kinase [Source:HGNC Symbol;Acc:HGNC:3697]"


In [12]:
grch38 %>% 
  #filter(biotype=="protein_coding") %>% 
  select(ensgene, entrez, symbol, chr, start, end, description) %>% 
  head


ensgene,entrez,symbol,chr,start,end,description
ENSG00000000003,7105,TSPAN6,X,100627109,100639991,tetraspanin 6 [Source:HGNC Symbol;Acc:HGNC:11858]
ENSG00000000005,64102,TNMD,X,100584802,100599885,tenomodulin [Source:HGNC Symbol;Acc:HGNC:17757]
ENSG00000000419,8813,DPM1,20,50934867,50958555,"dolichyl-phosphate mannosyltransferase subunit 1, catalytic [Source:HGNC Symbol;Acc:HGNC:3005]"
ENSG00000000457,57147,SCYL3,1,169849631,169894267,SCY1 like pseudokinase 3 [Source:HGNC Symbol;Acc:HGNC:19285]
ENSG00000000460,55732,C1orf112,1,169662007,169854080,chromosome 1 open reading frame 112 [Source:HGNC Symbol;Acc:HGNC:25565]
ENSG00000000938,2268,FGR,1,27612064,27635277,"FGR proto-oncogene, Src family tyrosine kinase [Source:HGNC Symbol;Acc:HGNC:3697]"


In [ ]:
grch38 %>% 
  filter(biotype=="protein_coding" & chr=="1") %>% 
  select(ensgene, symbol, chr, start, end, description) %>% 
  head %>% 
  pander::pandoc.table(split.table=100, justify="llllll", style="rmarkdown")

In [13]:
save(grch38, file = "./data_grch38.RData")

In [15]:
from_ref <- "symbol"
to_ref <- "ensgene"
id_df <- data.frame(raw_id = grch38$symbol[1:5])
colnames(id_df) <- from_ref

In [17]:
output_df <- id_df %>%
    left_join_(grch38, by = (from_ref = to_ref))

ERROR: Error in left_join_(., grch38, by = (from_ref = to_ref)): could not find function "left_join_"


In [24]:
by_str

ensgene 
"symbol"

In [32]:
by_str <- rlang::set_names(quo_name(from_ref), quo_name(from_ref))
#by_str <- rlang::set_names(quo_name(to_ref), quo_name(from_ref))
output_df <- id_df %>%
    left_join(grch38, by = by_str) #%>%
    #select_(to_ref)


Warning message:
“Column `symbol` joining factor and character vector, coercing into character vector”

In [33]:
output_df

symbol,ensgene,entrez,chr,start,end,strand,biotype,description
TSPAN6,ENSG00000000003,7105,X,100627109,100639991,-1,protein_coding,tetraspanin 6 [Source:HGNC Symbol;Acc:HGNC:11858]
TNMD,ENSG00000000005,64102,X,100584802,100599885,1,protein_coding,tenomodulin [Source:HGNC Symbol;Acc:HGNC:17757]
DPM1,ENSG00000000419,8813,20,50934867,50958555,-1,protein_coding,"dolichyl-phosphate mannosyltransferase subunit 1, catalytic [Source:HGNC Symbol;Acc:HGNC:3005]"
SCYL3,ENSG00000000457,57147,1,169849631,169894267,-1,protein_coding,SCY1 like pseudokinase 3 [Source:HGNC Symbol;Acc:HGNC:19285]
C1orf112,ENSG00000000460,55732,1,169662007,169854080,1,protein_coding,chromosome 1 open reading frame 112 [Source:HGNC Symbol;Acc:HGNC:25565]


In [35]:
sum(complete.cases(grch38[,1:3]))

[1] 28457

In [ ]:
id_df <- data.frame(raw_id = grch38$symbol[1:5])
colnames(id_df) <- from_ref
by_str <- rlang::set_names(quo_name(from_ref), quo_name(from_ref))
output_df <- id_df %>%
    left_join(annotable, by = by_str) #%>%
    #select_(to_ref)



In [44]:
class(grch38$ensgene)

[1] "character"

In [51]:
id_map <- function(id_vec, from_ref, to_ref, annotable = grch38) {
    id_df <- data.frame(raw_id = id_vec)
    colnames(id_df) <- from_ref
    id_df[, from_ref] <- as.character(id_df[, from_ref])
    by_str <- rlang::set_names(quo_name(from_ref), quo_name(from_ref))
    output_df <- id_df %>%
        left_join(annotable, by = by_str) %>%
        select_(to_ref)
    return(output_df[,1])                              
    
} 

In [41]:
class(id_map(grch38$symbol[4:9], from_ref = "symbol", to_ref = "ensgene"))

Warning message:
“Column `symbol` joining factor and character vector, coercing into character vector”

[1] "character"

In [52]:
id_map(grch38$symbol[4:9], from_ref = "symbol", to_ref = "ensgene")

[1] "ENSG00000000457" "ENSG00000000460" "ENSG00000000938" "ENSG00000000971"
[5] "ENSG00000001036" "ENSG00000001084"

In [53]:
id_map(grch38$symbol[4:9], from_ref = "symbol", to_ref = "entrez")

[1] 57147 55732  2268  3075  2519  2729

In [49]:
test_df <- head(grch38)

In [54]:
source("gene_id.R")

In [55]:
id_map(grch38$symbol[4:9], from_ref = "symbol", to_ref = "ensgene")

[1] "ENSG00000000457" "ENSG00000000460" "ENSG00000000938" "ENSG00000000971"
[5] "ENSG00000001036" "ENSG00000001084"

In [ ]:
id_map(grch38$ensgene[4:9], from_ref =, to_ref =  "symbol""ensgene")